In [27]:
import pandas as pd

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
data_paths = '/content/drive/MyDrive/aspects/top_aspects/'

In [30]:
# importing the module
import json
import ast
import re
topic = 'Gender'
all_counts = {}

for word_index_year in range(2014,2020):
  # reading the data from the file
  with open(data_paths + 'aspects_cnt' + topic + str(word_index_year) + '.txt') as f:
      data = f.read()

  data = ast.literal_eval(data)
  all_counts[word_index_year] = data

In [31]:
years = len(all_counts.keys())
get_param = (years * 90) / 100

ymain = list(all_counts.keys())[0]

pattern = [] # ['ooooo','oiiiii','ooioo','iiioo','iiiii','ioioi','ioooo','oioio','ooioo','iiooo']
def printAllKLength(set, k):
    printAllKLengthRec(set, "", k)

def printAllKLengthRec(set, string, k):
    if len(string) == k:
        pattern.append(string)
        return
    for c in set:
        printAllKLengthRec(set, string + c, k)
    return

set1 = ['I','O']
printAllKLength(set1, years)

print(len(pattern))

years = str(int(years))
get_param = str(int(get_param))

print(years,get_param)

def unique_word(myStr):
  pt = myStr.count('I')
  if pt == 1:
    return True
  return False

def IV_word(myStr):
  pt = re.fullmatch(r'^I{'+ years +'}',myStr) #re.search("^a")
  if pt:
    return True
  return False

def dead(myStr):
  pt = re.fullmatch(r'^I{1,'+ get_param +'}(?:|O{2,})$',myStr) #re.search("^a")
  if pt:
    return True
  return False

def born(myStr):
  pt = re.fullmatch(r'^O{2,'+ get_param +'}(?:|I{1,})$',myStr) #re.search("^a")
  if pt:
    return True
  return False

words_years = {}
unique_words = {}

union_words = set([k for k, v in all_counts[ymain].items()])
for y in all_counts.keys():
    words = set([k for k, v in all_counts[y].items()])
    words_years[y] = words
    union_words = union_words.union(words)

# len(union_words)

fre_words_dict = {}
for w in list(union_words):
    freq_row = []
    for y in all_counts.keys():
        if w in words_years[y]:
            freq_row.append('I')
        else:
            freq_row.append('O')
    # print(w, 'freq row is ',freq_row)
    fre_words_dict[w] = ''.join(freq_row)

IVs = 0
Deads = 0
Borns = 0
Seasonal = 0
Unique_ = 0

vocab_IVs = []
vocab_Deads = []
vocab_Borns = []
vocab_Seasonal = []
vocab_Unique_ = []

pattern_IVs = []
pattern_Deads = []
pattern_Borns = []
pattern_Seasonal = []
pattern_Unique_ = []

for k, v in fre_words_dict.items():
    if IV_word(v):
        IVs += 1
        vocab_IVs.append(k)
        pattern_IVs.append(v)
    elif unique_word(v):
        Unique_ += 1
        vocab_Unique_.append(k)
        pattern_Unique_.append(v)
    elif born(v):
        Borns += 1
        vocab_Borns.append(k)
        pattern_Borns.append(v)
    elif dead(v):
        Deads += 1
        vocab_Deads.append(k)
        pattern_Deads.append(v)
    else:
        Seasonal += 1
        vocab_Seasonal.append(k)
        pattern_Seasonal.append(v)

print(IVs,Unique_, Deads, Borns, Seasonal)

rows = []
for y in all_counts.keys():
  words =  set([k for k,v in all_counts[y].items()])

  words_IVs = words.intersection(vocab_IVs)
  words_Deads = words.intersection(vocab_Deads)
  words_Borns = words.intersection(vocab_Borns)
  words_Seasonal = words.intersection(vocab_Seasonal)
  words_Unique_ = words.intersection(vocab_Unique_)


  IVs = len(words_IVs)
  Deads = len(words_Deads)
  Borns = len(words_Borns)
  Seasonal = len(words_Seasonal)
  Unique_ = len(words_Unique_)

  rows.append([str(y), IVs,Unique_, Deads, Borns, Seasonal])

dataset = pd.DataFrame(rows,columns= ['Year', 'IVs','Unique_', 'Deads', 'Borns', 'Seasonal'])
dataset.to_csv(topic + '_' + 'aspects_statistics.csv') 
dataset 

64
6 5
321 89674 385 935 6224


,Year,IVs,Unique_,Deads,Borns,Seasonal
0,2014,321,11440,385,0,2112
1,2015,321,13183,385,0,2572
2,2016,321,11384,99,85,2521
3,2017,321,15184,33,272,3174
4,2018,321,24569,0,935,3527
5,2019,321,13914,0,935,2137


In [32]:
import operator
IVs = {k:v for k,v in all_counts[2014].items() if k in words_IVs}
# max(IVs , key=operator.itemgetter(1))
','.join(sorted(IVs, key=operator.itemgetter(1),reverse=True))

'hypocrite,hypocrisy,hypocritical,excellent point,excellent article,twice as likely,awesome,even know what,evil,funny,due,guilty,human right,due process,funny thing,sure,outrageous,but true,just as bad,turn blind eye,pure,strong,strict punishment,strange,stupid,it right,its good,other news,as bad as,ask a question,true,free,or false,free pass,or supreme,draconian law,criminal,great work,great,or top,wrong,crazy,or bad,free speech,true equality,true story,great article,great video,prove guilty,great piece,great job,great way,great example,great read,gross,break loose,break the law,great man,ironic,great discussion,equal right,special privilege,domestic abuse,good luck,wow,for false rape,top,good,woman empowerment,go to hell,good way,so true,you miss it,good news,good day,toxic masculinity,modern feminism,good job,good thing,sorry,so important,possible,powerful woman,good read,good example,corrupt,correct,not fair,wonder how many,for free,not surprised,you know what I,toxic,good man,hot,

# sorted by high frequency

In [33]:
rows = []
for y in all_counts.keys():
  words =  set([k for k,v in all_counts[y].items()])

  words_IVs = words.intersection(vocab_IVs)
  words_Deads = words.intersection(vocab_Deads)
  words_Borns = words.intersection(vocab_Borns)
  words_Seasonal = words.intersection(vocab_Seasonal)
  words_Unique_ = words.intersection(vocab_Unique_)

  flag = True
  n = 10
  l = 1
  IVs = {k:v for k,v in all_counts[y].items() if k in words_IVs and len(k) > n} #list(words_IVs)[0:5]
  IVs = list(sorted(IVs, key=operator.itemgetter(1),reverse=flag))[0:l]

  Deads = {k:v for k,v in all_counts[y].items() if k in words_Deads and len(k) > n} #list(words_Deads)[0:5]
  Deads = list(sorted(Deads, key=operator.itemgetter(1),reverse=flag))[0:l]

  Borns = {k:v for k,v in all_counts[y].items() if k in words_Borns and len(k) > n} #list(words_Borns)[0:5]
  Borns = list(sorted(Borns, key=operator.itemgetter(1),reverse=flag))[0:l]

  Seasonal = {k:v for k,v in all_counts[y].items() if k in words_Seasonal and len(k) > n} #list(words_Seasonal)[0:5]
  Seasonal = list(sorted(Seasonal, key=operator.itemgetter(1),reverse=flag))[0:l]

  Unique = {k:v for k,v in all_counts[y].items() if k in words_Unique_ and len(k) > n} #list(words_Unique_)[0:5]
  Unique = list(sorted(Unique, key=operator.itemgetter(1),reverse=flag))[0:l]

  rows.append([str(y), ','.join(IVs), ','.join(Unique), ','.join(Deads), ','.join(Borns), ','.join(Seasonal)])

dataset = pd.DataFrame(rows,columns= ['Year', 'IVs','Unique_', 'Deads', 'Borns', 'Seasonal'])
dataset.to_csv(topic + '_' + 'aspects_analysis'+ str(flag) + str(n) +'_' + str(l) +'.csv',index=None, encoding='utf-8-sig') 
dataset 

,Year,IVs,Unique_,Deads,Borns,Seasonal
0,2014,hypocritical,n’t want say,my mom say I,,by false accusation
1,2015,hypocritical,n’t concerned about the human right,my favorite,,systematic discrimination
2,2016,hypocritical,n’t sexual harassment allegation,punish court,such suicide,by vindictive woman
3,2017,hypocritical,y’all see how,just tell I,by sexual assault,by false allegation
4,2018,hypocritical,n’t partisan,,by sexual assault,systematic abuse
5,2019,hypocritical,n’t accuse disgrace,,my good wish,by false case


## sorted by low frequency

In [34]:
rows = []
for y in all_counts.keys():
  words =  set([k for k,v in all_counts[y].items()])

  words_IVs = words.intersection(vocab_IVs)
  words_Deads = words.intersection(vocab_Deads)
  words_Borns = words.intersection(vocab_Borns)
  words_Seasonal = words.intersection(vocab_Seasonal)
  words_Unique_ = words.intersection(vocab_Unique_)

  flag = False
  n = 10
  l = 1
  IVs = {k:v for k,v in all_counts[y].items() if k in words_IVs and len(k) > n} #list(words_IVs)[0:5]
  IVs = list(IVs)[0:l]

  Deads = {k:v for k,v in all_counts[y].items() if k in words_Deads and len(k) > n} #list(words_Deads)[0:5]
  Deads = list(Deads)[0:l]

  Borns = {k:v for k,v in all_counts[y].items() if k in words_Borns and len(k) > n} #list(words_Borns)[0:5]
  Borns = list(Borns)[0:l]

  Seasonal = {k:v for k,v in all_counts[y].items() if k in words_Seasonal and len(k) > n} #list(words_Seasonal)[0:5]
  Seasonal = list(Seasonal)[0:l]

  Unique = {k:v for k,v in all_counts[y].items() if k in words_Unique_ and len(k) > n} #list(words_Unique_)[0:5]
  Unique = list(Unique)[0:l]

  rows.append([str(y), ','.join(IVs), ','.join(Unique), ','.join(Deads), ','.join(Borns), ','.join(Seasonal)])

dataset = pd.DataFrame(rows,columns= ['Year', 'IVs','Unique_', 'Deads', 'Borns', 'Seasonal'])
dataset.to_csv(topic + '_' + 'aspects_analysis'+ str(flag) + str(n) +'_' + str(l) +'.csv',index=None, encoding='utf-8-sig') 
dataset 

,Year,IVs,Unique_,Deads,Borns,Seasonal
0,2014,interesting,really good movie,just tell I,,stupid thing
1,2015,interesting,inflict more atrocity,man be always,,more atrocity
2,2016,the good thing,cool bracelet,in abusive relationship,be available,mental patient
3,2017,be important,I just hope people,clear of rape,for sexual harassment,significant number
4,2018,right direction,Men stainless,,support all victim,in good fun
5,2019,turn blind eye,Top great value,,need national attention,super power
